<p style="font-family: Arial; font-size:2.75em;color:blue; font-style:bold"><br>
Projet 6 :<br><br>
Catégorisez automatiquement des questions<br><br>
</p><br>

* **Data Source / Data Source :** 
    - Data :
        - Outils d'extraction de data : https://data.stackexchange.com/stackoverflow
        - Script : https://github.com/EricJacquesPro/TextCategorization/blob/master/SQL/ExtractionDonnees.sql
    - Source Python : https://github.com/EricJacquesPro/TextCategorization
* **Description / Description : ** Projet 6 de la formation Data Scientist Proposée par OpenClassroom
    - Analyse LDA
    - Entrainement LDA
* **Auteur / Author : ** Eric JACQUES 
* **Date : ** 16/06/2020 

In [1]:
import sys
import os
import numpy as np
import pandas as pd
sys.path.insert(0, './Python/')

In [2]:
'''
python class for the data engineering (reading, cleaning, training...)
localised in ./Python
'''
from tagText import TagText 

C:\Users\naru_\Anaconda3\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
tagText = TagText()
print(tagText.urlDirectory)#folder for cvs file generated by SQL query (cf. I) 
print(tagText.fileName)#name of the cvs file generated by SQL query (cf. I)
data_question = tagText.read_source()
data_question.head()

tagText.nombre_post_entree = 500
tagText.precision = 500

Data/
QuestionVsTags.csv


In [4]:
data_preprocessed = data_question['body'].apply(tagText.preprocessing)
print(data_preprocessed[1:9])
type(data_preprocessed[1:9])

1    accidentally committed wrong files git pushed ...
2    want delete branch locally remotely failed att...
3                       differences git pull git fetch
4    ive messing around json time pushing text hurt...
5    use yield keyword python do example im trying ...
6    mistakenly added files git using command git a...
7    reading hidden features dark corners cstl comp...
8    want rename remote branch described rename mas...
Name: body, dtype: object


pandas.core.series.Series

In [5]:
print(data_preprocessed.values[0:])

[ 'piece c code shows peculiar behavior strange reason sorting data miraculously makes code almost six times faster include algorithm include ctime include iostream int main   generate data const unsigned arraysize  32768 int dataarraysize unsigned c  0 c  arraysize c datac  stdrand  256   this next loop runs faster stdsortdata data  arraysize  test clockt start  clock long long sum  0 unsigned  0  100000 i   primary loop unsigned c  0 c  arraysize c  datac  128 sum  datac   double elapsedtime  staticcastdoubleclock  start  clockspersec stdcout  elapsedtime  stdendl stdcout  sum    sum  stdendl  without stdsortdata data  arraysize code runs 1154 seconds sorted data code runs 193 seconds initially thought might language compiler anomaly tried java import javautilarrays import javautilrandom public class main  public static void mainstring args   generate data int arraysize  32768 int data  new intarraysize random rnd  new random0 int c  0 c  arraysize c datac  rndnextint  256   this nex

In [6]:
list(data_preprocessed)

['piece c code shows peculiar behavior strange reason sorting data miraculously makes code almost six times faster include algorithm include ctime include iostream int main   generate data const unsigned arraysize  32768 int dataarraysize unsigned c  0 c  arraysize c datac  stdrand  256   this next loop runs faster stdsortdata data  arraysize  test clockt start  clock long long sum  0 unsigned  0  100000 i   primary loop unsigned c  0 c  arraysize c  datac  128 sum  datac   double elapsedtime  staticcastdoubleclock  start  clockspersec stdcout  elapsedtime  stdendl stdcout  sum    sum  stdendl  without stdsortdata data  arraysize code runs 1154 seconds sorted data code runs 193 seconds initially thought might language compiler anomaly tried java import javautilarrays import javautilrandom public class main  public static void mainstring args   generate data int arraysize  32768 int data  new intarraysize random rnd  new random0 int c  0 c  arraysize c datac  rndnextint  256   this next

<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>
III.2 - Data - Bag of words - TF-IDF</p>

<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>
IV.2-a - Data - Tag generator - Unsupervised - LDA</p>

In [7]:
nb_topic_min = 5
nb_topic_max = 11#81
nb_topic_step = 2

In [8]:
perfs, perf_validation = tagText.lda_find_topic_number(
                                data_preprocessed,
                                nb_topic_min,
                                nb_topic_max,
                                nb_topic_step
                            )


ValueError: At least one array required as input

In [9]:
import matplotlib.pyplot as plt
import numpy as np
mini=min(perfs)
#print(perfs)
#print(perfs-mini)
x = range(nb_topic_min, nb_topic_max, nb_topic_step)

from sklearn.preprocessing import scale
perfs_normalized = scale( perfs, axis=0, with_mean=True, with_std=True, copy=True )
x_normalized = scale( x, axis=0, with_mean=True, with_std=True, copy=True )

plt.bar(x, perfs)
plt.ylabel("Performance")
plt.xlabel("number of topics")
plt.title("performance by number of topics")
plt.show()

plt.bar(x, perfs_normalized)
plt.ylabel("Performance normalized")
plt.xlabel("number of topics")
plt.title("performance normalized by number of topics")
plt.show()

perfs_topic_optimized = np.add(x_normalized, perfs_normalized)
plt.bar(x, perfs_topic_optimized)
plt.ylabel("Performance normalized + number of topic normalized")
plt.xlabel("number of topics")
plt.title("performance optimized by number of topics")
plt.show()

NameError: name 'perfs' is not defined

In [ ]:
    def lda_predict(
        model,
        text,
        lda,
        lda_df_topic_keywords,
        lda_tf_vectorizer,
        no_top_words
    ):
        '''
        predict tag form text in function of lda, topic ad tf vectorizer
        '''
        text = [text]
        mytext = lda_tf_vectorizer.transform(text)
        lda_topic_probability_scores = lda.transform(mytext)
        lda_topic = lda_df_topic_keywords.iloc[
            tagText.np.argmax(lda_topic_probability_scores),
            :
        ].values.tolist()
        topic_array = model.np.array(lda_topic)
        lda_feature_names = lda_tf_vectorizer.get_feature_names()
        """
        print(lda_topic)
        print(topic_array.argsort())
        print(lda_feature_names)
        """
        return (
            " ".join(
                        [
                            lda_feature_names[i]
                            for i in topic_array.argsort()
                            [
                                :-no_top_words - 1:-1
                            ]
                        ]
                )
        )

In [ ]:
lda_tagText, df_topic_keywords, tf_vectorizer = tagText.lda_prepare_tag(data_preprocessed, 45)

print(lda_predict(tagText, 'I cannot intall a git repository. I try github and gitlab', lda_tagText, df_topic_keywords, tf_vectorizer, 5))
print(tagText.lda_predict( "How to cast number as integer in java", lda_tagText, df_topic_keywords, tf_vectorizer, 5))

In [ ]:

print(lda_predict(tagText, 'I cannot intall a git repository. I try github and gitlab', lda_tagText, df_topic_keywords, tf_vectorizer, 5))

In [ ]:

print(tagText.lda_predict( "How to cast number as integer in java", lda_tagText, df_topic_keywords, tf_vectorizer, 5))

In [ ]:

print(tagText.lda_predict( "How to cast number as integer in java", lda_tagText, df_topic_keywords, tf_vectorizer, 10))